In [141]:
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd
# import time

# # Load match report links from file
# with open('match_report_links.txt', 'r') as file:
#     match_report_links = [line.strip() for line in file.readlines()]

# # List to store data for all players
# all_players_data = []

# # Function to extract player statistics from a table
# def extract_player_stats(table):
#     players_data = []
#     rows = table.find('tbody').find_all('tr')
#     print(f"Found {len(rows)} rows in the table.")
#     for row in rows:
#         player_data = {}
#         player_data['name'] = row.find('th', {'data-stat': 'player'}).text.strip()
#         player_data['pos'] = row.find('td', {'data-stat': 'position'}).text.strip()
#         player_data['minutes'] = row.find('td', {'data-stat': 'minutes'}).text.strip()
#         player_data['goals'] = row.find('td', {'data-stat': 'goals'}).text.strip()
#         player_data['assists'] = row.find('td', {'data-stat': 'assists'}).text.strip()
#         player_data['pens_made'] = row.find('td', {'data-stat': 'pens_made'}).text.strip()
#         player_data['pens_att'] = row.find('td', {'data-stat': 'pens_att'}).text.strip()
#         player_data['shots'] = row.find('td', {'data-stat': 'shots'}).text.strip()
#         player_data['shots_on_target'] = row.find('td', {'data-stat': 'shots_on_target'}).text.strip()
#         player_data['cards_yellow'] = row.find('td', {'data-stat': 'cards_yellow'}).text.strip()
#         player_data['cards_red'] = row.find('td', {'data-stat': 'cards_red'}).text.strip()
#         player_data['fouls'] = row.find('td', {'data-stat': 'fouls'}).text.strip()
#         player_data['fouled'] = row.find('td', {'data-stat': 'fouled'}).text.strip()
#         player_data['offsides'] = row.find('td', {'data-stat': 'offsides'}).text.strip()
#         player_data['crosses'] = row.find('td', {'data-stat': 'crosses'}).text.strip()
#         player_data['tackles_won'] = row.find('td', {'data-stat': 'tackles_won'}).text.strip()
#         player_data['interceptions'] = row.find('td', {'data-stat': 'interceptions'}).text.strip()
#         player_data['own_goals'] = row.find('td', {'data-stat': 'own_goals'}).text.strip()
#         player_data['pens_won'] = row.find('td', {'data-stat': 'pens_won'}).text.strip()
#         player_data['pens_conceded'] = row.find('td', {'data-stat': 'pens_conceded'}).text.strip()
#         players_data.append(player_data)
#         print(f"Extracted data for player: {player_data['name']}")
#     return players_data

# # Iterate through each match report link
# for link in match_report_links:
#     response = requests.get(link)
#     soup = BeautifulSoup(response.content, 'html.parser')

#     # Print raw HTML content of the page for debugging
#     print(f"Raw HTML content for {link}:\n")
#     print(soup.prettify()[:2000])  # Print the first 2000 characters for inspection

#     # Find all tables on the page
#     tables = soup.find_all('table')

#     # Print number of tables found for debugging
#     print(f"Found {len(tables)} tables in the page.")

#     # Extract player statistics from each table
#     for table in tables:
#         if table.find('caption') and 'Player Stats Table' in table.find('caption').text:
#             players_data = extract_player_stats(table)
#             all_players_data.extend(players_data)

#     # Delay to avoid hitting the rate limit
#     time.sleep(3)

# # Create DataFrame from the combined data
# df_players = pd.DataFrame(all_players_data)

# # Print column names for debugging
# print("Column names in DataFrame:", df_players.columns.tolist())

# # Clean up data (convert numerical columns from strings to integers)
# numerical_columns = ['minutes', 'goals', 'assists', 'pens_made', 'pens_att', 'shots', 'shots_on_target', 'cards_yellow', 'cards_red', 'fouls', 'fouled', 'offsides', 'crosses', 'tackles_won', 'interceptions', 'own_goals', 'pens_won', 'pens_conceded']
# for column in numerical_columns:
#     if column in df_players.columns:
#         df_players[column] = pd.to_numeric(df_players[column], errors='coerce').fillna(0).astype(int)
#     else:
#         print(f"Column {column} is missing in the DataFrame.")

# # Save the DataFrame to a CSV file
# df_players.to_csv('players_stats.csv', index=False)

# # Display the first few rows of the DataFrame
# print(df_players.head())


In [157]:
import pandas as pd

# Load the data into a pandas DataFrame
data = pd.read_csv('players_stats.csv')

# Fill missing values with 0
data = data.fillna(0)

# Aggregate the data by summing the numeric columns for each player
aggregated_data = data.groupby('name').sum(numeric_only=True).reset_index()

# For positions, we'll take the unique positions each player has been listed as
positions = data.groupby('name')['pos'].apply(lambda x: ','.join(set(x))).reset_index()

# Merge the aggregated numeric data with the unique positions
aggregated_data = pd.merge(aggregated_data, positions, on='name')

# Reorder columns to place 'pos' after 'name'
cols = ['name', 'pos'] + [col for col in aggregated_data.columns if col not in ['name', 'pos']]
aggregated_data = aggregated_data[cols]

# Order by minutes
aggregated_data = aggregated_data.sort_values(by='minutes', ascending=False)

# Save the aggregated data to a new CSV file
aggregated_data.to_csv('aggregated_player_stats.csv', index=False)


In [160]:
import pandas as pd
import re
# Function to clean player names
def clean_name(name):
    clean_name = re.split(r'\n', name)[0]
    return clean_name.strip()

def calculate_fantasy_score(row, match_difficulty=None):
    pos = row['pos']
    score = 0
    
    # Goals
    if 'FW' in pos:
        score += row['Performance Gls'] * 125000
    elif 'MF' in pos:
        score += row['Performance Gls'] * 150000
    elif 'DF' in pos:
        score += row['Performance Gls'] * 175000
    elif 'GK' in pos:
        score += row['Performance Gls'] * 250000
    
    # Assists
    score += row['Performance Ast'] * 60000
    
    # Yellow and Red Cards
    score -= row['Performance CrdY'] * 20000
    score -= row['Performance CrdR'] * 50000

    # Goalkeeper specific statistics
    if 'GK' in pos:
        score += row['saves'] * 5000  # Award points for saves
        score -= row['goals_against'] * 10000  # Deduct points for goals against
        score += row['save%'] * 1000  # Award points based on save percentage
    
    # Adjust score based on match difficulty if provided
    if match_difficulty:
        score *= match_difficulty.get(row['nation'], 1)
    
    # Weight score by the likelihood of starting (total minutes / 720)
    weight = row['minutes'] / 720
    print(row['name'], weight, row['minutes'])
    score *= weight
    
    return score

# Function to calculate multipliers from odds
def calculate_multipliers(odds):
    probabilities = {key: 1 / value for key, value in odds.items()}
    total = sum(probabilities.values())
    normalized_probabilities = {key: value / total for key, value in probabilities.items()}
    multipliers = {key: 1 / value for key, value in normalized_probabilities.items()}
    return multipliers

# Parse and store match odds
match_odds = {
    'GER': {'win': 1.30, 'draw': 6.00, 'lose': 12.50},
    'SCO': {'win': 12.50, 'draw': 6.00, 'lose': 1.30},
    'HUN': {'win': 3.50, 'draw': 3.50, 'lose': 2.20},
    'CHE': {'win': 2.20, 'draw': 3.50, 'lose': 3.50},
    'ESP': {'win': 1.78, 'draw': 3.85, 'lose': 5.00},
    'HRV': {'win': 5.00, 'draw': 3.85, 'lose': 1.78},
    'ITA': {'win': 1.39, 'draw': 4.85, 'lose': 9.70},
    'ALB': {'win': 9.70, 'draw': 4.85, 'lose': 1.39},
    'POL': {'win': 5.20, 'draw': 4.15, 'lose': 1.70},
    'NLD': {'win': 1.70, 'draw': 4.15, 'lose': 5.20},
    'SVN': {'win': 5.00, 'draw': 3.75, 'lose': 1.82},
    'DNK': {'win': 1.82, 'draw': 3.75, 'lose': 5.00},
    'SRB': {'win': 7.00, 'draw': 4.65, 'lose': 1.50},
    'ENG': {'win': 1.50, 'draw': 4.65, 'lose': 7.00},
    'ROU': {'win': 3.70, 'draw': 3.60, 'lose': 2.17},
    'UKR': {'win': 2.17, 'draw': 3.60, 'lose': 3.70},
    'BEL': {'win': 1.51, 'draw': 4.55, 'lose': 6.80},
    'SVK': {'win': 6.80, 'draw': 4.55, 'lose': 1.51},
    'AUT': {'win': 7.00, 'draw': 4.75, 'lose': 1.50},
    'FRA': {'win': 1.50, 'draw': 4.75, 'lose': 7.00},
    'TUR': {'win': 1.80, 'draw': 3.85, 'lose': 4.90},
    'GEO': {'win': 4.90, 'draw': 3.85, 'lose': 1.80},
    'PRT': {'win': 1.57, 'draw': 4.40, 'lose': 7.00},
    'CZE': {'win': 7.00, 'draw': 4.40, 'lose': 1.57}
}


In [172]:
# Calculate match difficulty multipliers
match_difficulty = {}
for team, odds in match_odds.items():
    multipliers = calculate_multipliers(odds)
    match_difficulty[team] = multipliers['win']

# Load the cleaned player prices
df_prices = pd.read_csv('cleaned_player_prices.csv')

# Apply the clean_name function to the name column
df_prices['name'] = df_prices['name'].apply(clean_name)

# Ensure names are standardized
df_prices['name'] = df_prices['name'].str.strip().str.title()

# Initialize FBRef data source for the Big 5 European Leagues combined
import soccerdata as sd
fbref = sd.FBref(leagues="Big 5 European Leagues Combined", seasons="2022-2023")

# Get player season statistics
player_season_stats = fbref.read_player_season_stats(stat_type="standard")

# Convert to DataFrame
df_stats = pd.DataFrame(player_season_stats)

# Reset the index to flatten the multi-level index
df_stats.reset_index(inplace=True)

# Flatten the MultiIndex columns
df_stats.columns = [' '.join(col).strip() for col in df_stats.columns.values]

# Rename columns to ensure clarity and consistency
df_stats.rename(columns={'player': 'name'}, inplace=True)

# Ensure names are standardized and matched correctly
df_stats['name'] = df_stats['name'].str.strip().str.title()

# Define the list of nationalities to include
nationalities_to_include = ['GER', 'ESP', 'ITA', 'NLD', 'ENG', 'BEL', 'FRA', 'PRT', 'DEN', 'TUR', 'UKR', 'CHE' ]
df_stats = df_stats[(df_stats['nation'].isin(nationalities_to_include))]

# Load Euro Qualifying data
df_qualifying = pd.read_csv('aggregated_player_stats.csv')

# Clean and standardize the names in the qualifying data
df_qualifying['name'] = df_qualifying['name'].apply(clean_name)
df_qualifying['name'] = df_qualifying['name'].str.strip().str.title()

# Merge qualifying data with the Big 5 data on player name and position
df_combined = pd.merge(df_stats, df_qualifying[['name', 'minutes', 'saves', 'goals_against', 'save%']], on='name', how='inner')

# Merge with price data
df_combined = pd.merge(df_combined, df_prices, on='name', how='inner')

# Enforce the rule that Antoine Griezmann is only considered a forward
df_combined.loc[df_combined['name'] == 'Antoine Griezmann', 'pos'] = 'FW'
df_combined.loc[df_combined['name'] == 'Kingsley Coman', 'pos'] = 'FW'
df_combined.loc[df_combined['name'] == 'Yannick Carrasco', 'pos'] = 'MF'
df_combined.loc[df_combined['name'] == 'Cengiz Ünder', 'pos'] = ''

# Update the fantasy score calculation to use the total minutes
df_combined['fantasy_score_with_difficulty'] = df_combined.apply(lambda row: calculate_fantasy_score(row, match_difficulty), axis=1)

# Calculate value for money (score) with match difficulty
df_combined['score_with_difficulty'] = df_combined['fantasy_score_with_difficulty'] / df_combined['Price']

# Function to select top players by position
def select_top_players(df, score_column):
    selected_team = pd.DataFrame()
    positions = {'GK': 1, 'DF': 3, 'MF': 4, 'FW': 3}
    
    for pos, count in positions.items():
        top_players = df[df['pos'].str.contains(pos)].sort_values(by=[score_column, 'minutes'], ascending=[False, False])
        selected_team = pd.concat([selected_team, top_players.head(count)])
        
        # Print the count of players for the current position
        print(f"Total players available for position {pos}: {df[df['pos'].str.contains(pos)].shape[0]}")
        print(f"Selected top {count} players for position {pos}:")
        print(top_players.head(count)[['name', 'score_with_difficulty']])
    
    # Ensure no more than 4 players from the same nation
    while selected_team['nation'].value_counts().max() > 4:
        for nation in selected_team['nation'].value_counts().index:
            if selected_team[selected_team['nation'] == nation].shape[0] > 4:
                lowest_scored_player = selected_team[selected_team['nation'] == nation].nsmallest(1, score_column)
                position = lowest_scored_player['pos'].values[0]
                next_best_player = df[(df['pos'].str.contains(position)) & (~df['name'].isin(selected_team['name'])) & (df['nation'] != nation)].nsmallest(1, 'Price')
                if next_best_player.empty:
                    continue
                selected_team = selected_team.drop(lowest_scored_player.index)
                selected_team = pd.concat([selected_team, next_best_player])

    return selected_team

# Example usage
selected_team_with_difficulty = select_top_players(df_combined, 'score_with_difficulty')

# Function to replace the lowest scored player with the next best cheaper player until the budget is within 50 million
def enforce_budget(selected_team, score_column, df):
    total_cost = selected_team['Price'].sum()
    
    while total_cost > 50_000_000:
        total_cost_millions = total_cost / 1_000_000
        print(f"Total cost is still over 50m, cost: {total_cost_millions:.2f}")
        
        # Find the lowest scored player
        sorted_team = selected_team.sort_values(by=score_column)
        
        for idx, lowest_scored_player in sorted_team.iterrows():
            print(f"Dropping lowest scored player: {lowest_scored_player['name']} with score {lowest_scored_player[score_column]} and price {lowest_scored_player['Price']}")
            
            position = lowest_scored_player['pos']
            
            # Find the next best cheaper player in the same position
            next_best_player = df[(df['pos'].str.contains(position)) & 
                                  (~df['name'].isin(selected_team['name'])) & 
                                  (df['Price'] < lowest_scored_player['Price'])].nlargest(1, score_column)
            
            if not next_best_player.empty:
                print(f"Adding next best player: {next_best_player['name'].values[0]} with score {next_best_player[score_column].values[0]} and price {next_best_player['Price'].values[0]}")
                
                selected_team = selected_team.drop(idx)
                selected_team = pd.concat([selected_team, next_best_player])
                total_cost = selected_team['Price'].sum()
                break
        else:
            # If no replacement found, attempt to drop the next lowest scored player
            print("No suitable replacement found with lower price and higher score. Trying next lowest scored player.")
            continue
        
        total_cost = selected_team['Price'].sum()
    
    return selected_team

# Example usage
selected_team_with_difficulty = select_top_players(df_combined, 'score_with_difficulty')

# Ensure at least 10 players for each position
def ensure_minimum_players(df, pos_column='pos', min_players=10):
    positions = ['GK', 'DF', 'MF', 'FW']
    additional_players = []

    for pos in positions:
        pos_players = df[df[pos_column].str.contains(pos)]
        if len(pos_players) < min_players:
            additional_needed = min_players - len(pos_players)
            additional_pos_players = df[~df.index.isin(pos_players.index) & df[pos_column].str.contains(pos)].head(additional_needed)
            additional_players.append(additional_pos_players)
    
    if additional_players:
        df = pd.concat([df] + additional_players).drop_duplicates()

    return df

# Ensure at least 10 players for each position
df_combined = ensure_minimum_players(df_combined)

# Select teams
selected_team_with_difficulty = select_top_players(df_combined, 'score_with_difficulty')

# Check the total cost for the team
total_cost_with_difficulty = selected_team_with_difficulty['Price'].sum()

# Enforce budget constraints
selected_team_with_difficulty = enforce_budget(selected_team_with_difficulty, 'score_with_difficulty', df_combined)

# Create the summary DataFrame
selected_team_summary = selected_team_with_difficulty[['name', 'Price', 'score_with_difficulty', 'pos']].sort_values(by=['score_with_difficulty'], ascending=[False])

# Save the summary to a new CSV file
selected_team_summary.to_csv('thewinners.csv', index=False)

# Display the summary
print("Selected Team with Difficulty:")
print(selected_team_summary)
print(f"Total Cost with Difficulty: {total_cost_with_difficulty:.2f} million")


[05/29/24 22:05:29] INFO     Saving cached data to C:\Users\Kasper\soccerdata\data\FBref              ]8;id=434458;file://c:\Users\Kasper\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=426806;file://c:\Users\Kasper\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#92\92]8;;\

Bukayo Saka 0.6805555555555556 490
Jorginho 0.2763888888888889 199
Leandro Trossard 0.40694444444444444 293
Oleksandr Zinchenko 0.8152777777777778 587
William Saliba 0.1638888888888889 118
Leander Dendoncker 0.11527777777777778 83
Ollie Watkins 0.07916666666666666 57
Illia Zabarnyi 1.125 810
Philip Billing 0.05555555555555555 40
Christian Nørgaard 0.5513888888888889 397
David Raya 0.25 180
Ivan Toney 0.013888888888888888 10
Mathias Jensen 0.42083333333333334 303
Leandro Trossard 0.40694444444444444 293
Conor Gallagher 0.13194444444444445 95
Jorginho 0.2763888888888889 199
Mykhailo Mudryk 0.9972222222222222 718
Joachim Andersen 0.875 630
Amadou Onana 0.48194444444444445 347
Jordan Pickford 2.0 1440
Vitaliy Mykolenko 1.1458333333333333 825
James Maddison 0.30277777777777776 218
Timothy Castagne 0.9652777777777778 695
Wout Faes 1.0 720
Youri Tielemans 0.5652777777777778 407
Trent Alexander-Arnold 0.49027777777777776 353
Aymeric Laporte 0.625 450
Cole Palmer 0.05138888888888889 37
Jack Gre